In [62]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [93]:
url = 'https://www.booking.com/searchresults.html?label=gog235jc-1DCAEoggI46AdICVgDaFCIAQGYAQm4ARfIAQzYAQPoAQH4AQKIAgGoAgO4ArDuuaEGwAIB0gIkZmJhYjE4YzAtNDdhMy00MmY1LTk2NWItN2UzOTgyNTk1OWEx2AIE4AIB&aid=397594&ss=bangkok&ssne=Taipei&ssne_untouched=Taipei&lang=en-us&sb=1&src_elem=sb&src=searchresults&dest_id=-3414440&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=84c942d8c6c30191&ac_meta=GhA4NGM5NDJkOGM2YzMwMTkxIAAoATICZW46A2JhbkAASgBQAA%3D%3D&checkin=2023-12-06&checkout=2023-12-07&ltfd=6%3A1%3A%3A%3A1&group_adults=2&no_rooms=1&group_children=0&offset=975'
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
    'Accept-Language': 'en-US, en;q=0.5'
}

response  = requests.get(url, headers=headers)

In [94]:
soup = bs(response.text, 'html.parser')

In [ ]:
https://www.booking.com/searchresults.html?label=gog235jc-1DCAEoggI46AdICVgDaFCIAQGYAQm4ARfIAQzYAQPoAQH4AQKIAgGoAgO4ArDuuaEGwAIB0gIkZmJhYjE4YzAtNDdhMy00MmY1LTk2NWItN2UzOTgyNTk1OWEx2AIE4AIB&aid=397594&ss=London&ssne=London&ssne_untouched=London&lang=en-us&sb=1&src_elem=sb&src=searchresults&dest_id=-2601889&dest_type=city&checkin=2023-12-06&checkout=2023-12-07&ltfd=6%3A1%3A5-2023%3A&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure&selected_currency=USD&soz=1&lang_changed=1&offset=0
https://www.booking.com/searchresults.html?label=gog235jc-1DCAEoggI46AdICVgDaFCIAQGYAQm4ARfIAQzYAQPoAQH4AQKIAgGoAgO4ArDuuaEGwAIB0gIkZmJhYjE4YzAtNDdhMy00MmY1LTk2NWItN2UzOTgyNTk1OWEx2AIE4AIB&aid=397594&ss=bangkok&ssne=Taipei&ssne_untouched=Taipei&lang=en-us&sb=1&src_elem=sb&src=searchresults&dest_id=-3414440&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=84c942d8c6c30191&ac_meta=GhA4NGM5NDJkOGM2YzMwMTkxIAAoATICZW46A2JhbkAASgBQAA%3D%3D&checkin=2023-12-06&checkout=2023-12-07&ltfd=6%3A1%3A%3A%3A1&group_adults=2&no_rooms=1&group_children=0&offset=975

In [109]:
def scrape_pages(soup):
    hotels = soup.findAll('div', {'data-testid': 'property-card'})
    hotel_data=[]

    for hotel in hotels:
        name_elements = hotel.find('div', {'data-testid': 'title'})
        name = name_elements.text.strip()
        # name_lst.append(name)

        location_element = hotel.find('span', {'data-testid': 'address'})
        location = location_element.text.strip()
        # loc_lst.append(location)

        price_element = hotel.find('span', {'data-testid': 'price-and-discounted-price'})
        price = price_element.text.strip()
        # price_lst.append(price)

        rating_element = hotel.find('div', {'class': 'a3b8729ab1 d86cee9b25'})
        try:
            rating = rating_element.text.strip()
        except:
            rating = pd.NA
        # rating_lst.append(rating)

        # distance
        distance_element = hotel.find('span', {'data-testid': 'distance'})
        distance = distance_element.text.strip()
        # dis_lst.append(distance)

        # comment
        comment_element = hotel.find('div', {'class': 'a3b8729ab1 e6208ee469 cb2cbb3ccb'})

        try:
            comment = comment_element.text.strip()
        except:
            comment = pd.NA
        # com_lst.append(comment)

        hotel_data.append({
            'name': name,
            'location': location,
            'price': price,
            'rating': rating,
            'distance': distance,
            'comment': comment
        })
    return hotel_data


In [114]:
items_element = soup.find('div', {'class': 'd8f77e681c'})
items_count = int(items_element.text.strip().split(' ')[-3])

hotel_df = pd.DataFrame()

for item in range(0, items_count, 25):
    
    # print(item)
    # booking 只能顯示最多1000 筆資料
    if item < 1000:
        searchURL = url + '&offset=' + str(item)

        page_response = requests.get(searchURL, headers=headers)
        page_soup = bs(page_response.text, 'html.parser')

        df = pd.DataFrame(scrape_pages(page_soup))
        hotel_df = pd.concat([hotel_df, df], axis=0)
    else:
        break
    

In [115]:
hotel_df

,name,location,price,rating,distance,comment
0,Chatrium Hotel Riverside Bangkok,"Bang Kho Laem, Bangkok","TWD 6,019",8.9,5.1 km from center,Excellent
1,"Marriott Executive Apartments Bangkok, Sukhumv...","Wattana, Bangkok","TWD 4,811",8.8,8.9 km from center,Excellent
2,Studio Ekamai,"Wattana, Bangkok","TWD 1,020",7.7,9.9 km from center,Good
3,HOMM Sukhumvit34 Bangkok a brand of BANYAN TRE...,"Khlong Toei, Bangkok","TWD 2,398",8.0,8 km from center,Very Good
4,P-Park Hotel near Yanhee Hospital - SHA Extra ...,Bangkok,"TWD 1,087",7.0,5.6 km from center,Good
...,...,...,...,...,...,...
20,Zayn Express & Suites,Bangkok,TWD 977,7.0,14.9 km from center,Good
21,Marsi Hotel @ Bangchak,"Sukhumvit, Bangkok",TWD 945,8.7,12.6 km from center,Excellent
22,Merry V Guesthouse,"Khaosan, Bangkok",TWD 665,7.2,1.4 km from center,Good
23,Orion Rooms Sukhumvit 33,"Wattana, Bangkok",TWD 617,7.0,7.1 km from center,Good


In [116]:
def data_clean(hotel_df):
    # 去除文字
    hotel_df['price'] = hotel_df['price'].str.lstrip('TWD\xa0')
    hotel_df['distance'] = hotel_df['distance'].str.rstrip(' km from center').astype(float)

    # 去除分隔符
    hotel_df['price'] = hotel_df['price'].str.replace(',','').astype(int)
    hotel_df['rating'] = hotel_df['rating'].astype(float)

    # 去除空值
    hotel_df.dropna(axis=0, how='any', inplace=True)

    return hotel_df


,name,location,price,rating,distance,comment
0,Chatrium Hotel Riverside Bangkok,"Bang Kho Laem, Bangkok",6019,8.9,5.1,Excellent
1,"Marriott Executive Apartments Bangkok, Sukhumv...","Wattana, Bangkok",4811,8.8,8.9,Excellent
2,Studio Ekamai,"Wattana, Bangkok",1020,7.7,9.9,Good
3,HOMM Sukhumvit34 Bangkok a brand of BANYAN TRE...,"Khlong Toei, Bangkok",2398,8.0,8.0,Very Good
4,P-Park Hotel near Yanhee Hospital - SHA Extra ...,Bangkok,1087,7.0,5.6,Good
...,...,...,...,...,...,...
20,Zayn Express & Suites,Bangkok,977,7.0,14.9,Good
21,Marsi Hotel @ Bangchak,"Sukhumvit, Bangkok",945,8.7,12.6,Excellent
22,Merry V Guesthouse,"Khaosan, Bangkok",665,7.2,1.4,Good
23,Orion Rooms Sukhumvit 33,"Wattana, Bangkok",617,7.0,7.1,Good


In [ ]:
hotel_df = data_clean(hotel_df)

In [122]:
# Visualization
import plotly.express as px

fig = px.scatter(hotel_df, x='price', y='distance', color='rating')
fig.show()
